![Image](images/actinia_logo.png)

## Kartierung von Überflutungsgebieten aus Sentinel-1 Radardaten mit Hilfe von actinia

### Die actinia Prozesskette

Actinia nutzt den **Prozesskettenansatz**, um für Import, Verarbeitung und
Export von Geodaten mit dem actinia GRASS GIS Verarbeitungssystem zu 
kommunizieren. Die Prozesskette muss in JSON formuliert sein.

Eine Prozesskette ist eine Liste von GRASS GIS Modulen, die nacheinander in der Reihenfolge der Liste ausgeführt werden. GRASS GIS Module werden als Prozessdefinitionen spezifiziert, die den Namen des Befehls, die Ein- und Ausgaben, einschließlich Import- und Exportdefinitionen sowie die Modulflags enthalten.

Dieser Workshop benutzt die Python Bibliothek **requests** für Interaktion mit dem actinia REST Dienst.

---
### actinia API Dokumentation

* [actinia "stable" API v3 docs](https://redocly.github.io/redoc/?url=https://actinia.mundialis.de/api/v3/swagger.json)
* [actinia "development" API v3 docs](https://redocly.github.io/redoc/?url=https://actinia-dev.mundialis.de/api/v3/swagger.json)

---
### Anforderungen

#### Software & Module

Dieses Tutorial setzt voraus, dass Sie mit der Programmiersprache [Python](https://python.org) vertraut sind. Die Kenntnis grundlegender [REST](https://de.wikipedia.org/wiki/Representational_State_Transfer)-API-Konzepte und deren Verwendung wird ebenfalls vorausgesetzt.

Die in diesem Tutorium verwendeten Python-Module sind:
* [requests](http://docs.python-requests.org/)
* [json](https://docs.python.org/3/library/json.html)
* [leafmap](https://leafmap.org/)
* [actinia-python-client](https://actinia-org.github.io/actinia-python-client/)


#### ACTINIA API Benutzer und Passwort

Für diesen Workshop werden die Anmeldedaten für die Authentifizierung benötigt, die unten in **Vorbereitung** als Variable festgelegt sind. Eine andere actinia-Instanz kann andere Anmeldedaten erfordern.

### Python Module und Hilfs-Funktionen

Bevor wir mit dem actinia-Server über Python interagieren, werden wir die erforderlichen Pakete importieren und eine Hilfsfunktion einrichten, um ein formatiertes JSON ausgeben zu lassen.

## Vorbereitung

Wir nutzen [Leafmap](https://leafmap.org) um die Ergebnisse von actinia zu visualisieren. Leafmap ist ein Python-Paket für interaktives Mapping und raumbezogene Analysen mit minimalem Programmieraufwand in einer Jupyter-Umgebung (siehe auch: [actinia with leafmap notebook](https://github.com/actinia-org/actinia-python-client/blob/main/notebooks/actinia_leafmap.ipynb)).

Als weiteres Paket installieren wir den Python Client Library für actinia ([actinia-python-client](https://github.com/actinia-org/actinia-python-client/)).

**Wichtig:** Danach müssen wir uns einmal ausloggen (im Menü File -> Log Out) und wieder einloggen.

In [ ]:
!pip install -U leafmap
!pip install actinia-python-client==0.4.1

Bitte einmal ausloggen (im Menü File -> Log Out) und wieder einloggen.

Im mundialis JupyterHub müssen danach unter Umständen Pfade neu gesetzt werden, damit matplotlib auch gefunden wird. Eventuelle Warnungen von pip install müssen berücksichtigt werden.

In [ ]:
import os
import pwd
import sys

user = pwd.getpwuid(os.getuid()).pw_name
pythonversion = f"{sys.version_info[0]}.{sys.version_info[1]}"

# adjust PATH
pythonbin = f"/home/{user}/.local/bin"
path = os.getenv("PATH")
path = path + os.pathsep + pythonbin
os.environ["PATH"] = path

# adjust PYTHONPATH

pythonpath = f"/home/{user}/.local/lib/python{pythonversion}/site-packages"
sys.path.append(pythonpath)

In [ ]:
# Zuerst werden die erforderlichen Bibliotheken importiert.

import json

import leafmap
import requests
from requests.auth import HTTPBasicAuth
from actinia.utils import create_actinia_pc_item

Um die Kommunikation mit dem actinia Server zu vereinfachen, speichern wir die Anmeldedaten und die URL des REST-Servers in Variablen:

In [ ]:
# variables to set the actinia host, version, and user

actinia_baseurl = "https://actinia.mundialis.de"
actinia_version = "v3"
actinia_user = "fossgis2025"
actinia_pw = "mohtaiF9Tu5fohw7"
actinia_url = f"{actinia_baseurl}/api/{actinia_version}"
# user, pw for the FOSSGIS 2023 workshop
actinia_auth = HTTPBasicAuth(actinia_user, actinia_pw)

## Hilfs-Funktionen

In [ ]:
# helper function to print formatted JSON using the json module

def print_as_json(data):
    print(json.dumps(data, indent=2))

# helper function to verify a request
def verify_request(request, success_code=200):
    if request.status_code != success_code:
        print("ERROR: actinia processing failed with status code %d!" % request.status_code)
        print("See errors below:")
        print_as_json(request.json())
        request_url = request.json()["urls"]["status"]
        requests.delete(url=request_url, auth=actinia_auth)
        raise Exception("The resource <%s> has been terminated." % request_url)

## Detektion von Wasserflächen mit Sentinel-1 Radardaten

### Eingabedaten

In diesem Beispiel werden Sentinel-1 Radardaten als Input für die Detektion von Wasserflächen, z.B. Überflutungsgebieten verwendet. Radardaten haben gegenüber optischen Aufnahmen den Vorteil, dass aktive Sensoren wie Sentinel-1 Radardaten aussenden und die Rückstrahlung (backscatter) aufnehmen und somit auch unabhängig von Wolkenbedeckung und Sonnenschein sind.

Sentinel-1 ist ein aktiver C-Band-SAR-Satellit (Synthetic Aperture Radar) und war die erste Satellitenserie, die im Rahmen des Copernicus-Programms gestartet wurde. Diese Serie besteht aus mehreren Einzelsatelliten, Sentinel-1A, Sentinel-1B, Sentinel-1C und Sentinel-1D, was zu hohen potenziellen Wiederholungszeiten von ein bis fünf Tagen je nach Standort führt. Von diesen vier Satellten sind aktuell Sentinel-1A und Sentinel-1C aktiv. Sentinel-1B wurde abgeschaltet, Sentinel-1D startet Ende 2025. Wir verwenden in diesem Workshop GRD (ground-range detected) Produkte mit einer räumlichen Auflösung von 10 m. 

### Wasserflächen im Radarbild

Zusätzlich zu Messungen des Meeresspiegels können aus den Sentinel-1 Daten verschiedene andere Bodeneigenschaften abgeleitet werden, z.B. Bodenfeuchte, Rauhigkeit, Waldbeckung, Eisbdeckung, Wasserbedeckung. Wasser und Land lassen sich theoretisch recht einfach unterscheiden, da Wasser niedrigere Werte hat als Land. Wenn für eine gegebene Landfläche eine bimodale Verteilung der Werte vorliegt, kann davon ausgegangen werden, dass größere Flächen von offenem Wasser bedeckt sind.

<div>
<img src="images/water_land_bimodal-gr3_lrg.jpg" width="600"/>
</div>

### Zusätzliche Eingabedaten

Zusätzlich wird ein Höhenmodell verwendet, um für jeden Pixel die Höhe über dem nahesten Wasserabfluss zu bestimmen. Diese Information wird benutzt, um falsch klassifizierte Wasserpixel zu entfernen.


## Vorbereitung von Sentinel-1 Radardaten

SAR-Daten (Synthetic Aperture Radar), wie sie von Sentinel-1 geliefert werden, sind sehr komplex und erfordern umfangreiche Vorverarbeitung, um für eigentliche Analysen geeignet zu sein. Zu den typischen Schritten gehören die Kalibrierung zur Beseitigung von Sensorrauschen, die radiometrische Geländekorrektur zur Anpassung an topografische Effekte und die Orthorektifizierung zur Gewährleistung der räumlichen Genauigkeit. Diese Schritte sind unerlässlich, um Radar-Rohdaten in "analyse-ready Daten" (ARD) umzuwandeln, die anschließend mit geografischen Analysetools verwendet werden können.

Seit kurzem werden Sentinel-1 Daten auch als Analysis-Ready-Data (ARD) bereitgestellt, allerdings nur als monatliche Komposite. Diese monatliche Aggregierung ist allerdings nicht geeignet, um kurzfrsitige Überflutungsereignisse zu erfassen.

Üblicherweise werden Sentinel-1 Rohdaten mit ESA's Open Source [SNAP](https://step.esa.int/main/download/snap-download/) Software aufbereitet.

Für GRASS gibt es das addon [i.sentinel1.pyrosargeocode](https://github.com/NVE/actinia_modules_nve/tree/main/src/imagery/i.sentinel1.pyrosargeocode), das auch in aktuellen actinia images enthalten ist. Dieses addon benutzt zusätzlich zu SNAP auch [pyroSAR](https://github.com/johntruckenbrodt/pyroSAR) Der Prozess der Umwandlung von SAR-Daten ist jedoch sehr rechenintensiv, deswegen wurden die in diesem Workshop benötigten Sentinel-1 Daten bereits vorverarbeitet und stehen in der benutzten actinia Instanz zur Verfügung.


#### Mapsets in Locations auflisten

Auflisten aller **Mapsets** innerhalb der Location `fossgis2025_ecuador` über den Endpunkt `/locations/<location_name>/mapsets`:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/fossgis2025_ecuador/mapsets"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Mapsets in fossgis2025_ecuador location:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

#### Inhalt einer Mapset auflisten

Auflisten aller **Rasterkarten** in der Location `fossgis2025_ecuador` und Mapset `S1_watermask` über den Endpunkt `/locations/<location_name>/mapsets/<mapset>/raster_layers`:

In [ ]:
# make a GET request to the actinia data API
request_url = f"{actinia_url}/locations/fossgis2025_ecuador/mapsets/S1_watermask/raster_layers"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Raster layers in mapset S1_watermask of location fossgis2025_ecuador:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

Abfrage der **Rasterinformationen** von der Karte `S1B_IW_GRDH_1SDV_20170330T233603_20170330T233634_004944_008A54_8BDD_Sigma0_VV_log` über den Endpunkt `/locations/<location_name>/mapsets/<mapset>/raster_layers/<raster>`:

In [ ]:
# make a GET request to the actinia data API
request_url = f"{actinia_url}/locations/fossgis2025_ecuador/mapsets/S1_watermask/raster_layers/S1B_IW_GRDH_1SDV_20170330T233603_20170330T233634_004944_008A54_8BDD_Sigma0_VV_log"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Raster info of sample S1 radar in mapset S1_watermask of location fossgis2025_ecuador:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

## Erstellen einer Prozesskette Schritt für Schritt

Zuerst wird die actinia request URL für asynchrones Prozessieren gesetzt und eine leere Prozesskette erstellt.

In [ ]:
# create a POST request to the Actinia Data API
request_url = f"{actinia_url}/locations/fossgis2025_ecuador/processing_async_export"

process_chain = {"version": 1, "list": []}

## Hinzufügen von Einträgen zu der Prozesskette

Setzen der aktuellen Region für Raster-Prozessierung

In [ ]:
# list item for g.region
list_id = "g_region_watermask"
# long form, as in a process chain
inputs = [{"param": "n",
           "value": "9847795"
          },
          {"param": "s",
           "value": "9806336"
          },
          {"param": "w",
           "value": "597713"
          },
          {"param": "e",
           "value": "653372"
          },
          {"param": "align",
           "value": "S1B_IW_GRDH_1SDV_20170330T233603_20170330T233634_004944_008A54_8BDD_Sigma0_VV_log@S1_watermask"
          }]

# short form, accepted by create_actinia_pc_item
inputs = {"n": "9847795",
          "s": "9806336",
          "w": "597713",
          "e": "653372",
          "align": "S1B_IW_GRDH_1SDV_20170330T233603_20170330T233634_004944_008A54_8BDD_Sigma0_VV_log@S1_watermask"}

flags = "p"
stdout = {"id": "region_watermask", "format": "list", "delimiter": "\n"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="g.region",
                                 inputs=inputs,
                                 stdout=stdout,
                                 flags=flags)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

### HAND (Height Above Nearest Drainage) vom Höhenmodell berechnen

#### Info zum DEM Raster

In [ ]:
# list item for r.info
list_id = "r_info_dem"
# long form
inputs = [{"param": "map",
           "value": "ecuador_nasadem_rounded@S1_watermask"
          }]

# short form
inputs = {"map": "ecuador_nasadem_rounded@S1_watermask"}
flags = "r"

stdout = {"id": "dem_range", "format": "kv", "delimiter": "="}
pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.info",
                                 inputs=inputs,
                                 flags=flags,
                                 stdout=stdout)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Flussnetzwerk berechnen

In [ ]:
# list item for r.watershed
list_id = "r_watershed_streams"
# long form
inputs = [{"param": "elevation",
           "value": "ecuador_nasadem_rounded@S1_watermask"
          },
          {"param": "threshold",
           "value": "10000"
          },
          {"param": "convergence",
           "value": "5"
          }]
outputs = [{"param": "drainage",
            "value": "drainage"
           },
           {"param": "stream",
            "value": "streams"
           }]

# short form
inputs = {"elevation": "ecuador_nasadem_rounded@S1_watermask",
          "threshold": "10000",
          "convergence": "5"}
outputs = {"drainage": "drainage",
           "stream": "streams"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.watershed",
                                 inputs=inputs,
                                 outputs=outputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Höhe über nahestem Fluss

In [ ]:
# list item for r.stream.distance
list_id = "r_stream_distance"
# long form
inputs = [{"param": "stream_rast",
           "value": "streams"
          },
          {"param": "direction",
           "value": "drainage"
          },
          {"param": "elevation",
           "value": "ecuador_nasadem_rounded@S1_watermask"
          },
          {"param": "method",
           "value": "downstream"
          }]
outputs = [{"param": "difference",
            "value": "HAND_ecuador_raw"
           }]

# short form
inputs = {"stream_rast": "streams",
          "direction": "drainage",
          "elevation": "ecuador_nasadem_rounded@S1_watermask",
          "method": "downstream"}
outputs = {"difference": "HAND_ecuador_raw"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.stream.distance",
                                 inputs=inputs,
                                 outputs=outputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Gefälle des Geländes

In [ ]:
# list item for r.slope.aspect
list_id = "r_slope_aspect"
# long form
inputs = [{"param": "elevation",
           "value": "ecuador_nasadem_rounded@S1_watermask"
          },
          {"param": "format",
           "value": "degrees"
          },
          {"param": "precision",
           "value": "FCELL"
          },
          {"param": "zscale",
           "value": "1.0"
          },
          {"param": "min_slope",
           "value": "0.0"
          }]
outputs = [{"param": "slope",
            "value": "ecuador_nasadem_rounded_slope"
           }]

# short form
inputs = {"elevation": "ecuador_nasadem_rounded@S1_watermask",
          "format": "degrees",
          "precision": "FCELL",
          "zscale": "1.0",
          "min_slope": "0.0"}
outputs = {"slope": "ecuador_nasadem_rounded_slope"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.slope.aspect",
                                 inputs=inputs,
                                 outputs=outputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### HAND Raster mit Gefälle anpassen

In [ ]:
# list item for r.mapcalc
list_id = "r_mapcalc_hand_with_slope"
# long form
inputs = [{"param": "expression",
           "value": "HAND_ecuador_slopes_included10 = if(ecuador_nasadem_rounded_slope >= 10, 100, HAND_ecuador_raw)"
          }]

# short form
inputs = {"expression": "HAND_ecuador_slopes_included10 = if(ecuador_nasadem_rounded_slope >= 10, 100, HAND_ecuador_raw)"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.mapcalc",
                                 inputs=inputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Sentinel-1 Raster zuschneiden

Durch das Zuschneiden des Sentinel-1 Rasters auf die aktuelle Region wird die Erstellung der Wassermaske erheblich beschleunigt.

In [ ]:
# list item for r.mapcalc
list_id = "r_mapcalc_clip_s1"
# long form
inputs = [{"param": "expression",
           "value": "S1_20170330_8BDD_VV_clip = S1B_IW_GRDH_1SDV_20170330T233603_20170330T233634_004944_008A54_8BDD_Sigma0_VV_log@S1_watermask"
          }]

# short form
inputs = {"expression": "S1_20170330_8BDD_VV_clip = S1B_IW_GRDH_1SDV_20170330T233603_20170330T233634_004944_008A54_8BDD_Sigma0_VV_log@S1_watermask"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.mapcalc",
                                 inputs=inputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Wassermaske erstellen

In [ ]:
# list item for i.sentinel_1.water
list_id = "i_sentinel_1_water"
# long form
inputs = [{"param": "input",
           "value": "S1_20170330_8BDD_VV_clip"
          },
          {"param": "hand_rast",
           "value": "HAND_ecuador_slopes_included10"
          },
          {"param": "polarization",
           "value": "VV"
          }]
outputs = [{"param": "output",
            "value": "S1_watermask_20170330_8BDD_VV_clip"
           }]

# short form
inputs = {"input": "S1_20170330_8BDD_VV_clip",
          "hand_rast": "HAND_ecuador_slopes_included10",
          "polarization": "VV"}
outputs = {"output": "S1_watermask_20170330_8BDD_VV_clip"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="i.sentinel_1.water",
                                 inputs=inputs,
                                 outputs=outputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Ergebnis exportieren

In [ ]:
# list item for the exporter
list_id = "export_watermask"
# long form
outputs = [{"export": {"type": "raster", "format": "GTiff"},
            "param": "map",
            "value": "S1_watermask_20170330_8BDD_VV_clip"}]

# short form not working here because of the special "export" entry

pc_item = create_actinia_pc_item(id=list_id,
                                 module="exporter",
                                 outputs=outputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Job submission

Die Prozesskette wird mit der POST Methode an actinia geschickt

In [ ]:
# submit the job
request = requests.post(url=request_url, auth=actinia_auth, json=process_chain)
# check if anything went wrong
verify_request(request, 200)

#### actinia Antwort

actinia schickt eine Antwort zurück mit Informationen, ob die Prozesskette akzeptiert wurde

In [ ]:
# get a json-encoded content of the response
jsonResponse = request.json()

print(f"Response with status code: {request.status_code}")

# print formatted JSON
print_as_json(jsonResponse)

#### Job Status

Der Status des actinia jobs wird abgefragt

In [ ]:
# make a GET request to the Actinia Data API
request_url = jsonResponse["urls"]["status"]
print("actinia GET request:")
print(request_url)
print("---")

request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

#### Job Ende

Der Status des actinia jobs wird weiter abgefragt, bis er erfolgreich beendet wurde oder auf einen Fehler läuft. Wenn der Job erfolgreich beendet wurde, steht in der actinia Antwort "Processing successfully finished"

In [ ]:
# continue polling until finished
while request.status_code == 200 and \
        jsonResponse["message"] != "Processing successfully finished":
    request = requests.get(url=request_url, auth=actinia_auth)
    jsonResponse = request.json()

# check if anything went wrong
verify_request(request, 200)

#### Job logs und Ergebnisse

actinia Antwort anzeigen

In [ ]:
# print formatted JSON
print_as_json(jsonResponse)

Prozessergebnisse ausgeben lassen:

In [ ]:
print("Process results:")
print_as_json(jsonResponse["process_results"])

Ergebnis-URL aus actinia Antwort selektieren und anzeigen.

In [ ]:
result_url = request.json()["urls"]["resources"][0]

print(result_url)
raster_url = result_url.replace("//", f"//{actinia_user}:{actinia_pw}@")

### Ergebnis hier im notebook anzeigen

In [ ]:
# visualization with leafmap
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
m.add_cog_layer(
    raster_url,
    label="S1 Wassermaske",
    colormap_name="set1",
)
legend_dict = {
    "0 Land": "e41a1c",
    "1 Wasser": "999999",
}
m.add_legend(
    title="Wassermaske", legend_dict=legend_dict, draggable=False
)
# show map
m